# Gist
Prototype selection is treated from a generative perspective here. A form for $p(X \mid y,S)$ is assumed (S is the set of prototypes, X denotes the features and y the labels) where the probability of each point depends only on its nearest neighbor having the same label. Using this assumption, we maximize $f(S) = log\lbrack p(S \mid X,y) \rbrack$  to find the set S of prototypes. Although this is a combinatorial optimization problem, we observe that f(S) is a submodular, monotone function. For this class of functions, approximate greedy inference has been both practically and theoretically shown to be useful. Here we use a greedy algorithm to maximize S.  

# Description of cost function
We take a more detailed look at the cost function in order to better understand the method. For some subset S of the input examples, let us consider the posterior probability of S.
$$p(S | X,y) ∝ p(X | y,S)p(y | S)p(S)$$  
$$\implies \log \lbrack p(S | X, y) \rbrack = \log p(X | y, S)  + \log p(y | S)  + \log p(S)  + c$$  
where c is a constant. We maximize this function assuming a specific form p(X | y,S) that satisfies two main conditions 
* It accurately captures the requirements for good classification of a nearest neighbour classifier
* The resulting cost function is submodular

To do this, we assume that the probability of a feature vector is related to the nearest neighbor which has the same label as that feature vector. Specifically, we assume
$$p(X | y, S) =   maxr∈Syi eα||xi−r||2 (3) i$$
where Syi represents the subset of S with label equal to yi.
Substituting this in and making benign rearrangements, we get the cost function
$$f(S) =   log p(xi | yi, S)  (4) i
   ||xi −xk||2 log p(yi |S)  
=⇒ f(S)=$$
where $\alpha$ and $\beta$ are hyper-parameters and $p(yi \mid S)$ is just the proportion of prototypes of label yi in set S. This function can be shown to be submodular. The proof could be done for example by showing that each p(xi | yi, S) is submodular and hence the sum is submodular.


In [1]:
# Load the dataset
%matplotlib inline
import cPickle, gzip, numpy
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from itertools import chain
import copy
import random
import sys
import functools
import scipy.spatial.distance as distance
import math
import cProfile
import numpy as np
import multiprocess
from multiprocessing import Manager,Lock
import numpy as np
from collections import defaultdict
from scipy.spatial.distance import hamming
import heapq
from collections import namedtuple
import unittest
import sklearn.metrics as metrics
import os

f = gzip.open('mnist.pkl.gz', 'rb')
train_set, valid_set, test_set = cPickle.load(f)
f.close()

train_x = 1*(train_set[0]>0.5)
test_x = 1*(test_set[0]>0.5)
valid_x = 1*(valid_set[0]>0.5)

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def fprime_manual(f,x,s,args = ()):
    """
    Manual calculation of marginal gain. If there is a optimal custom implementation
    this should be overidden by supplying the fprime argument. Similar in spirit to 
    numerical gradient calculation.
    
    Parameters:
    __________
    f: callable f(x,s,*args); the objective function
    x: element of input set to calculate marginal gain for
    args: tuple (optinal); additional arguments to be supplied to f
    
    Returns:
    _______
    Marginal gain for element x, set s for function f
    """
    if x in s:
        return 0
    else:
        s_new = copy.deepcopy(s)
        s_current = copy.deepcopy(s_new)
        
        s_new.append(x)
        return f(s_new,*args) - f(s_current,*args)

def fmax_submodular(f,s,fprime = None,sopt = None,k=float("Inf"),verbose = False,args=()):
    
    """
    Maximize a submodular function using the greedy method.
    
    Parameters:
    __________
    s: set of elements, search is performed over the powerset P(s).
    f: callable f(x,*args)
    fprime: callable fprime(x,s);If not supplied, defaults to manual cauculation
    args: tuple (optional) additional arguments to be passed to f
    k: Cardinatlity constraint |S|<k is enforced

    Returns:
    _______
    sopt: Optimal set for f
    fopt: Value of the function at the optimal point
    """
    
    if fprime == None:
        fprime = lambda x,s: fprime_manual(f,x,s,args)
    
    "Checking if solution class has all required interfaces"
    if sopt == None:
        sopt = []
    elif hasattr(sopt,"__len__") & hasattr(sopt,"append") & hasattr(sopt,"__contains__"):
        l = getattr(sopt,"__len__")
        ap = getattr(sopt,"append")
        in_func = getattr(sopt,"__contains__")
        
        if callable(l) & callable(ap) & callable(in_func):
            pass
        else:
            raise TypeError("Solution class improperly defined")
    else:
        raise TypeError("Solution class improperly defined")
        
    fopt = 0

    #Populate the heap
    heap_fprimes = [(-fprime(x,sopt),x) for x in s]
    heapq.heapify(heap_fprimes)
    
    #print heap_fprimes
    
    while len(sopt)<k:
        x = heapq.heappop(heap_fprimes)
        x = (-fprime(x[1],sopt),x[1])

        while len(heap_fprimes)>0 and x[0]>heap_fprimes[0][0]:
            heapq.heappush(heap_fprimes,x)
            x = heapq.heappop(heap_fprimes)
            x = (-fprime(x[1],sopt),x[1])

        if x[0]>=0:
            break
        else:
            sopt.append(x[1])
            fopt += -x[0]
        if verbose:
            print str(100*float(len(sopt))/float(k)) + "% done; Value of objective: "+str(fopt)
            print len(sopt)
            print f(sopt)
            
    solution = namedtuple("solution",["sopt","fopt"])
    
    return solution(sopt,fopt)

def set_cover(s):
	if len(s)==0:
		return 0
	else:
		return len(reduce(lambda x,y:x|y,s))+1

class testdf:
    def __init__(self):
        self.x =0

soln = fmax_submodular(set_cover,[{1,2},{1,3,4},{1,2,4,5,6},{7,8}],k = 4,verbose = True)

25.0% done; Value of objective: 6
1
6
50.0% done; Value of objective: 8
2
8
75.0% done; Value of objective: 9
3
9


In [3]:
class solution:
    
    def __init__(self,x,y):
        
        self.train_data = x
        self.train_labels = y
        self.label_mapping = defaultdict(list)
        self.distances = dict()
        self.solution = []
        self.data_currentdistances = [float(784)]*x.shape[0]
        self.solution_sizes = defaultdict(int)
        self.distance_matrices = dict()
        self.index_map = dict()
        
        for index,label in enumerate(train_set[1]):
            self.label_mapping[label].append(index)
            self.index_map[index] = len(self.label_mapping[label])-1
        
        for label in self.label_mapping:
            label_set = self.label_mapping[label]
            self.distance_matrices[label] = np.inner(x[label_set,],1-x[label_set,]) + np.inner(1-x[label_set,],x[label_set,])
            
    def append(self,ind):
        if ind in self.solution:
            return False
        else:
            for matrix_index,point_index in enumerate(self.label_mapping[self.train_labels[ind]]):
                new_dist = self.distance_matrices[self.train_labels[ind]][self.index_map[ind],matrix_index]
                self.data_currentdistances[point_index] = min(self.data_currentdistances[point_index],new_dist)
                
            self.solution.append(ind)
            self.solution_sizes[self.train_labels[ind]] += 1
            return True
    
    def clear_solution(self):
        self.solution = []
        self.data_currentdistances = [float(784)]*self.train_data.shape[0]
        self.solution_sizes = defaultdict(int)
        return True
        
    def __repr__(self):
        return str(self.solution)
    
    def __len__(self):
        return len(self.solution)
    
    def __contains__(self,item):
        return item in self.solution

#Getting the solution construction profile    
cProfile.run("s = solution(train_x,train_set[1])")

         100023 function calls in 281.882 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    2.681    2.681  281.882  281.882 <ipython-input-3-c5cd3c5b9fff>:3(__init__)
        1    0.000    0.000  281.882  281.882 <string>:1(<module>)
    50000    0.005    0.000    0.005    0.000 {len}
    50000    0.004    0.000    0.004    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
       20  279.193   13.960  279.193   13.960 {numpy.core.multiarray.inner}




In [4]:
def grad(ind,self,alpha = 0,beta = 0):
        
        delta = 0
        if ind in self.solution:
            return 0
        else:
                
            for matrix_index,point_index in enumerate(self.label_mapping[self.train_labels[ind]]):
                new_dist = self.distance_matrices[self.train_labels[ind]][self.index_map[ind],matrix_index]
                if self.data_currentdistances[point_index] > new_dist:
                    delta += self.data_currentdistances[point_index] - new_dist
                if self.solution_sizes[self.train_labels[ind]] != 0:
                    delta += alpha*math.log( (self.solution_sizes[self.train_labels[ind]]+1)/self.solution_sizes[self.train_labels[ind]] )

            delta = delta/self.train_data.shape[0] - beta
            
            if len(self.solution)>0:
                delta += alpha*math.log( float(len(self.solution))/(len(self.solution) + 1) )
        
            return delta

def objective_function(s,alpha = 0,beta = 0):
    """
    Arguments:
    _________
    s represents the dictionary containing the list of prototypes for each label
    """
    cost = -sum(s.data_currentdistances)
    
    
    for label in s.train_labels:
        try:
            cost += alpha*math.log(s.solution_sizes[label]/float(len(s.solution)))
        except ValueError:
            return float("-Inf")
            
    cost = cost/float(s.train_data.shape[0]) - beta*len(s.solution)
    
    return cost

def predict(sol,test_data):
    
    s = np.array(sol.solution)
    distances = np.dot(1-sol.train_data[s,:],np.transpose(test_data)) + np.dot(sol.train_data[s,:],1-np.transpose(test_data))
    dist_indexes = np.zeros((test_data.shape[0],),dtype=int)
    val = np.argmin(distances,axis=0,out =dist_indexes)
    
    return sol.train_labels[s[dist_indexes]]

print "Checking gradient correctness using numerical calculation"

count = 0
while count<10:
    s.clear_solution()
    randomIndices = np.random.randint(0,len(train_set[1]),100)
    map(s.append,randomIndices)

    g = lambda x: fprime_manual(objective_function,x,s)
    delta = np.random.randint(0,len(train_set[1]))
    
    count += 1
    print "Error is: " + str(abs(grad(delta,s) - g(delta)))

Checking gradient correctness using numerical calculation
Error is: 2.33146835171e-15
Error is: 7.77156117238e-16
Error is: 5.27355936697e-15
Error is: 1.12757025938e-14
Error is: 3.77475828373e-15
Error is: 1.19002030452e-14
Error is: 8.32667268469e-16
Error is: 7.91033905045e-15
Error is: 7.29277749301e-15
Error is: 3.44863027024e-15


In [5]:
randomIndices = np.random.randint(0,len(train_set[1]),100)

s.clear_solution()

#Profiling the prototype add time
cProfile.run("map(s.append,randomIndices)")

#Profiling the objective function evaluation time
cProfile.run('o = objective_function(s)')
print o

#Gradient evaluation time
cProfile.run("d = grad(110,s)")
print d

         506860 function calls in 0.654 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.509    0.005    0.654    0.007 <ipython-input-3-c5cd3c5b9fff>:23(append)
        1    0.000    0.000    0.654    0.654 <string>:1(<module>)
        1    0.000    0.000    0.654    0.654 {map}
      100    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
   506657    0.145    0.000    0.145    0.000 {min}


         100005 function calls in 0.052 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.040    0.040    0.052    0.052 <ipython-input-4-dc6880ac9ab0>:22(objective_function)
        1    0.000    0.000    0.052    0.052 <string>:1(<module>)
    50001    0.003    0.000    0.003    0.000 {len}
    50000    0.006    0.000    0.006    0

In [10]:
readings = []
s.clear_solution()

soln = namedtuple("soln",["sopt","fopt"])
sol_optim = soln(copy.deepcopy(s),0)

for subset_size in [50,100,200,500,1000,5000,10000]:

    print "Starting model learning"
    sol_optim = fmax_submodular(objective_function,range(len(train_set[1])),fprime = grad,sopt = sol_optim.sopt,k=subset_size,verbose = True)
    pred_test = predict(sol_optim.sopt,test_x)
    
    ac = metrics.accuracy_score(pred_test,test_set[1])
    pr = metrics.precision_recall_fscore_support(pred_test,test_set[1],average="macro")
    vlist = list(pr)
    vlist = ["Optim",ac,subset_size] + vlist[0:-1]
    readings.append(vlist)
    
    print "Done with model learning"
    
    trials = 0
    while trials<20:
        randomIndices = np.random.randint(0,len(train_set[1]),subset_size)
        s.clear_solution()
        map(s.append,randomIndices)
        pred_test = predict(s,test_x)

        ac = metrics.accuracy_score(pred_test,test_set[1])
        pr = metrics.precision_recall_fscore_support(pred_test,test_set[1],average="macro")
        vlist = list(pr)
        vlist = ["Random",ac,subset_size] + vlist[0:-1]
        readings.append(vlist)
        trials += 1
        
    print "Random model evaluation done"
    
with open('Readings.dat','w') as f:
    cPickle.dump(readings,f)

Starting model learning
2.0% done; Value of objective: 83.84404
1
-inf
4.0% done; Value of objective: 157.06422
2
-inf
6.0% done; Value of objective: 227.24598
3
-inf
8.0% done; Value of objective: 297.0655
4
-inf


KeyboardInterrupt: 

In [ ]:
import pandas as pd

data = pd.DataFrame(readings,columns=['Algorithm','Accuracy','SubsetSize','Precision','Recall','Fscore'])
print data
filtdat = data[data.Algorithm == 'Random']
ax1 = filtdat.plot(x = 'SubsetSize', y = 'Accuracy', kind = 'box')

#filtdat = data[data.Algorithm == 'Optim']
#ax2 = filtdat.plot(x = 'SubsetSize', y = 'Accuracy', kind = 'line')

plt.grid()
plt.show()

$x^{2}/2$ is the value